In [1]:
"""Compute depth maps for images in the input folder.
"""
import os
import glob
import torch
import utils
import cv2
import argparse
import time
import gc
import numpy as np

from imutils.video import VideoStream
from midas.model_loader import default_models, load_model

first_execution = True
from datasets import load_dataset
from tqdm import tqdm

In [2]:
dataset = load_dataset("Isamu136/big-animal-dataset-high-res-embedding")

Using custom data configuration Isamu136--big-animal-dataset-high-res-embedding-ccb1b56dcdb50954
Found cached dataset parquet (D:/cache/huggingface/datasets/Isamu136___parquet/Isamu136--big-animal-dataset-high-res-embedding-ccb1b56dcdb50954/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
def process(device, model, model_type, image, input_size, target_size, optimize, use_camera):
    """
    Run the inference and interpolate.

    Args:
        device (torch.device): the torch device used
        model: the model used for inference
        model_type: the type of the model
        image: the image fed into the neural network
        input_size: the size (width, height) of the neural network input (for OpenVINO)
        target_size: the size (width, height) the neural network output is interpolated to
        optimize: optimize the model to half-floats on CUDA?
        use_camera: is the camera used?

    Returns:
        the prediction
    """
    global first_execution

    if "openvino" in model_type:
        if first_execution or not use_camera:
            print(f"    Input resized to {input_size[0]}x{input_size[1]} before entering the encoder")
            first_execution = False

        sample = [np.reshape(image, (1, 3, *input_size))]
        prediction = model(sample)[model.output(0)][0]
        prediction = cv2.resize(prediction, dsize=target_size,
                                interpolation=cv2.INTER_CUBIC)
    else:
        sample = torch.from_numpy(image).to(device).unsqueeze(0)

        if optimize and device == torch.device("cuda"):
            if first_execution:
                print("  Optimization to half-floats activated. Use with caution, because models like Swin require\n"
                      "  float precision to work properly and may yield non-finite depth values to some extent for\n"
                      "  half-floats.")
            sample = sample.to(memory_format=torch.channels_last)
            sample = sample.half()

        if first_execution or not use_camera:
            height, width = sample.shape[2:]
            print(f"    Input resized to {width}x{height} before entering the encoder")
            first_execution = False

        prediction = model.forward(sample)
        prediction = (
            torch.nn.functional.interpolate(
                prediction.unsqueeze(1),
                size=target_size[::-1],
                mode="bicubic",
                align_corners=False,
            )
            .squeeze()
            .cpu()
            .numpy()
        )

    return prediction


def create_side_by_side(image, depth, grayscale):
    """
    Take an RGB image and depth map and place them side by side. This includes a proper normalization of the depth map
    for better visibility.

    Args:
        image: the RGB image
        depth: the depth map
        grayscale: use a grayscale colormap?

    Returns:
        the image and depth map place side by side
    """
    depth_min = depth.min()
    depth_max = depth.max()
    normalized_depth = 255 * (depth - depth_min) / (depth_max - depth_min)
    normalized_depth *= 3

    right_side = np.repeat(np.expand_dims(normalized_depth, 2), 3, axis=2) / 3
    if not grayscale:
        right_side = cv2.applyColorMap(np.uint8(right_side), cv2.COLORMAP_INFERNO)

    if image is None:
        return right_side
    else:
        return np.concatenate((image, right_side), axis=1)




In [8]:
def run(dataset, image_column, output_path, model_path, model_type="dpt_beit_large_512", optimize=False, side=False, height=None,
        square=False, grayscale=False):
    """Run MonoDepthNN to compute depth maps.

    Args:
        input_path (str): path to input folder
        output_path (str): path to output folder
        model_path (str): path to saved model
        model_type (str): the model type
        optimize (bool): optimize the model to half-floats on CUDA?
        side (bool): RGB and depth side by side in output images?
        height (int): inference encoder image height
        square (bool): resize to a square resolution?
        grayscale (bool): use a grayscale colormap?
    """
    print("Initialize")

    # select device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Device: %s" % device)

    model, transform, net_w, net_h = load_model(device, model_path, model_type, optimize, height, square)

    # get input
    num_images = len(dataset)


    # create output folder
    if output_path is not None:
        os.makedirs(output_path, exist_ok=True)

    print("Start processing")

    if output_path is None:
        print("Warning: No output path specified. Images will be processed but not shown or stored anywhere.")
    for index, example in tqdm(enumerate(dataset)):
        torch.cuda.empty_cache()
        gc.collect()
        image = example['image']
        if not image.mode == "RGB":
            image = image.convert("RGB")

        # input
        original_image_rgb = np.array(image)/255.
        image = transform({"image": original_image_rgb})["image"]

        # compute
        with torch.no_grad():
            prediction = process(device, model, model_type, image, (net_w, net_h), original_image_rgb.shape[1::-1],
                                 optimize, False)
        # output
        if output_path is not None:
            filename = os.path.join(
                output_path, str(index) + '-' + model_type
            )
            if not side:
                utils.write_depth(filename, prediction, grayscale, bits=2)
            else:
                original_image_bgr = np.flip(original_image_rgb, 2)
                content = create_side_by_side(original_image_bgr*255, prediction, grayscale)
                cv2.imwrite(filename + ".png", content)
            utils.write_pfm(filename + ".pfm", prediction.astype(np.float32))


    print("Finished")

In [9]:
image_column="image"
output_path = "segmented"
model_path = "weights/dpt_beit_large_512.pt"

In [10]:
run(dataset["train"], image_column, output_path, model_path, model_type="dpt_beit_large_512", optimize=False, side=False, height=None, square=False, grayscale=False)

Initialize
Device: cuda
Model loaded, number of parameters = 345M
Start processing


0it [00:00, ?it/s]

    Input resized to 512x352 before entering the encoder


1it [00:02,  2.16s/it]

    Input resized to 512x352 before entering the encoder


...

2774it [28:15,  1.90it/s]

    Input resized to 352x512 before entering the encoder


2775it [28:16,  1.64it/s]

    Input resized to 512x352 before entering the encoder


KeyboardInterrupt: 

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'caption', 'l14_embeddings', 'moco_vitb_imagenet_embeddings', 'moco_vitb_imagenet_embeddings_without_last_layer', 'ibot_b_16_embedding', 'ibot_b_16_last_self_attn'],
        num_rows: 26180
    })
})